In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import sqlite3
import io

# Define the FraudDetector model class
class FraudDetector(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Define the Autoencoder model class
class Autoencoder(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 16),
            nn.ReLU(),
            nn.Linear(16, 8)
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, input_size)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Function to upload and load dataset
def load_dataset():
    print("Please upload your CSV file (e.g., indian_fraud_dataset.csv)")
    uploaded = files.upload()  # Opens file picker in Colab
    if not uploaded:
        raise ValueError("No file uploaded.")
    file_name = list(uploaded.keys())[0]
    df = pd.read_csv(io.BytesIO(uploaded[file_name]))
    print(f"Dataset loaded with {len(df)} transactions from {file_name}.")
    return df, file_name

# Preprocess data for prediction
def preprocess_for_prediction(df, mean_amount, std_amount, loc_encoder, dev_encoder):
    df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
    df['loc_idx'] = df['location'].map(loc_encoder)
    df['dev_idx'] = df['device'].map(dev_encoder)
    df['loc_idx'] = df['loc_idx'].fillna(-1)
    df['dev_idx'] = df['dev_idx'].fillna(-1)
    df['amount_norm'] = (df['amount_inr'] - mean_amount) / std_amount
    features = ['amount_norm', 'hour', 'loc_idx', 'dev_idx']
    X = df[features].values
    return X, df

# Load trained models
def load_models(sup_model_path='supervised_fraud_model.pth', auto_model_path='autoencoder_model.pth', input_size=4):
    supervised_model = FraudDetector(input_size)
    supervised_model.load_state_dict(torch.load(sup_model_path))
    supervised_model.eval()
    autoencoder = Autoencoder(input_size)
    autoencoder.load_state_dict(torch.load(auto_model_path))
    autoencoder.eval()
    return supervised_model, autoencoder

# Simulate real-time fraud detection for a single transaction
def simulate_real_time(sup_model, autoencoder, trans, mean_amount, std_amount, loc_encoder, dev_encoder, threshold=0.5):
    hour = pd.to_datetime(trans['timestamp']).hour
    loc_idx = loc_encoder.get(trans['location'], -1)
    dev_idx = dev_encoder.get(trans['device'], -1)
    amount_norm = (trans['amount_inr'] - mean_amount) / std_amount
    features = np.array([amount_norm, hour, loc_idx, dev_idx])
    features_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        sup_pred = sup_model(features_tensor).item()
        recon = autoencoder(features_tensor)
        loss = torch.mean((recon - features_tensor)**2).item()
    is_anomaly = loss > threshold
    is_fraud = sup_pred > 0.5 or is_anomaly
    return is_fraud, sup_pred, is_anomaly

# Process dataset for real-time fraud monitoring
def real_time_monitoring(df, supervised_model, autoencoder, mean_amount, std_amount, loc_encoder, dev_encoder):
    fraud_detected = []
    print("Starting real-time fraud monitoring...")
    for idx, trans in df.iterrows():
        is_fraud, sup_pred, is_anomaly = simulate_real_time(
            supervised_model, autoencoder, trans, mean_amount, std_amount, loc_encoder, dev_encoder
        )
        if is_fraud:
            fraud_detected.append({
                'transaction_id': trans['transaction_id'],
                'customer_id': trans['customer_id'],
                'amount_inr': trans['amount_inr'],
                'timestamp': trans['timestamp'],
                'location': trans['location'],
                'device': trans['device'],
                'fraud_reason': f"Supervised score: {sup_pred:.4f}, Anomaly: {is_anomaly}"
            })
            print(f"ALERT: Fraud detected! Transaction ID: {trans['transaction_id']}, Customer: {trans['customer_id']}, Amount: ₹{trans['amount_inr']:.2f}")
    return pd.DataFrame(fraud_detected)

# Main function with file upload
def main(sup_model_path='supervised_fraud_model.pth', auto_model_path='autoencoder_model.pth'):
    # Step 1: Upload and load dataset
    df, file_name = load_dataset()

    # Step 2: Compute normalization parameters and encoders
    mean_amount = df['amount_inr'].mean()
    std_amount = df['amount_inr'].std()
    loc_encoder = {loc: idx for idx, loc in enumerate(df['location'].unique())}
    dev_encoder = {dev: idx for idx, dev in enumerate(df['device'].unique())}

    # Step 3: Load models
    supervised_model, autoencoder = load_models(sup_model_path, auto_model_path, input_size=4)

    # Step 4: Process transactions in real-time
    fraud_df = real_time_monitoring(df, supervised_model, autoencoder, mean_amount, std_amount, loc_encoder, dev_encoder)

    # Step 5: Output results
    if not fraud_df.empty:
        print("\nFraud Detected Transactions/People:")
        print(fraud_df[['transaction_id', 'customer_id', 'amount_inr', 'timestamp', 'location', 'device', 'fraud_reason']])
        fraud_df.to_csv('fraud_detected.csv', index=False)
        print("\nFraud detected data saved to 'fraud_detected.csv'.")

        # Save to SQLite
        conn = sqlite3.connect("fraud_detection_db.db")
        fraud_df.to_sql("fraud_transactions", conn, if_exists="replace", index=False)
        conn.commit()
        conn.close()
        print("Fraud transactions inserted into SQLite database.")
    else:
        print("No fraud detected in the dataset.")

if __name__ == "__main__":
    main()

Please upload your CSV file (e.g., indian_fraud_dataset.csv)


Saving indian_fraud_dataset.csv to indian_fraud_dataset (1).csv
Dataset loaded with 1000 transactions from indian_fraud_dataset (1).csv.
Starting real-time fraud monitoring...
ALERT: Fraud detected! Transaction ID: 8, Customer: Cust087, Amount: ₹1929950.81
ALERT: Fraud detected! Transaction ID: 9, Customer: Cust075, Amount: ₹2703.08
ALERT: Fraud detected! Transaction ID: 16, Customer: Cust053, Amount: ₹6333.29
ALERT: Fraud detected! Transaction ID: 48, Customer: Cust021, Amount: ₹4045.82
ALERT: Fraud detected! Transaction ID: 51, Customer: Cust018, Amount: ₹540.52
ALERT: Fraud detected! Transaction ID: 57, Customer: Cust090, Amount: ₹25081.47
ALERT: Fraud detected! Transaction ID: 68, Customer: Cust078, Amount: ₹495390.29
ALERT: Fraud detected! Transaction ID: 73, Customer: Cust002, Amount: ₹1035.68
ALERT: Fraud detected! Transaction ID: 84, Customer: Cust074, Amount: ₹70003.31
ALERT: Fraud detected! Transaction ID: 90, Customer: Cust015, Amount: ₹664001.23
ALERT: Fraud detected! Trans